# E. coli Promoter Classification with Genomic Ensemble Pretraining

This notebook follows on from the previous. Here we train a promoter classification model in the same way as before. The only thing we change is initializing the classification model with weights from a better language model. The previous language model was trained on just the E. coli genome. It trained to a validation loss of 2.63. The new model is trained on an ensemble of different bacterial genomes, which trained to a validation loss of 2.50. We will see that this has a large impact on performance.

To avoid clutter, confusion, and long notebooks, I separated the training of the new language model to a different notebook. The data preparation for the bacterial ensemble is found in the [Bacterial Ensemble Data Processing](https://github.com/kheyer/Genomic-ULMFiT/blob/master/Bacteria/Bacterial%20Ensemble/Genomic%20Language%20Models/Bacterial%20Ensemble%20LM%200%20Data%20Processing.ipynb) notebook. Training of the new language model is detailed in the [Bacterial Ensemble Language Model](https://github.com/kheyer/Genomic-ULMFiT/blob/master/Bacteria/Bacterial%20Ensemble/Genomic%20Language%20Models/Bacterial%20Ensemble%20LM%201%205-mer%20Language%20Model.ipynb) notebook.

This notebook shows how using a better trained language model as the basis for a classification model has a significant effect on performance - much more so than training longer on classification data. This method really shines when you have a lot of unlabeled genomic data to throw at it.

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
import sys
from pathlib import Path
import pandas as pd
import numpy as np
from typing import Callable

# sets device for model and PyTorch tensors
os.environ["CUDA_VISIBLE_DEVICES"]="0"
#os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3,4,5,6,7" # "-1" if want to run on a CPU, otherwise define the GPU number

In [3]:
from fastai import *
from fastai.text import ( BaseTokenizer, Tokenizer, Vocab, 
                         PreProcessor, ItemList, PathOrStr, 
                         DataFrame, Optional, Collection, 
                         IntsOrStrs, DataBunch, is_listy, 
                         ItemLists, TextList, SortishSampler, 
                         DataLoader, SortSampler, partial, 
                         pad_collate, TextLMDataBunch, data_collate, 
                         LanguageModelPreLoader )
from Bio import Seq
from Bio.Seq import Seq
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import FeatureLocation, CompoundLocation
import networkx as nx

In [4]:
sys.path.append("../..")
from utils import ( _get_genomic_processor, get_model_clas, TextClasDataBunch, 
                    get_scores, split_data, GenomicTokenizer, get_model_LM, 
                   GenomicVocab, GenomicTextClasDataBunch)

In [5]:
path = Path('/mnt/wd_4tb/shared_disk_wd4tb/mattscicluna/data/genomic_ulmfit/ecoli/')
path_bact = Path('/mnt/wd_4tb/shared_disk_wd4tb/mattscicluna/data/genomic_ulmfit/bacterial_genomes/')

# Classification

In [ ]:
classification_df = pd.read_csv(path/'e_coli_promoters_dataset.csv')

In [ ]:
classification_df.head()

In [ ]:
train_df = classification_df[classification_df.set == 'train']
valid_df = classification_df[classification_df.set == 'valid']
test_df = classification_df[classification_df.set == 'test']

Here we create a dataloader with the vocabulary used for the bacterial ensemble model

In [ ]:
voc = np.load(path_bact/'bact_vocab.npy')
model_vocab = GenomicVocab(voc)

In [ ]:
tok = Tokenizer(GenomicTokenizer, n_cpus=1, pre_rules=[], post_rules=[], special_cases=['xxpad'])
data_clas = GenomicTextClasDataBunch.from_df(path, train_df, valid_df, tokenizer=tok, vocab=model_vocab,
                                            text_cols='Sequence', label_cols='Promoter', bs=300)

The classification model is structured exactly the same as in previous notebooks

In [ ]:
clas_config = dict(emb_sz=400, n_hid=1150, n_layers=3, pad_token=0, qrnn=False, output_p=0.4, 
                       hidden_p=0.2, input_p=0.6, embed_p=0.1, weight_p=0.5)
drop_mult = 0.4

In [ ]:
learn = get_model_clas(data_clas, drop_mult, clas_config)

In [ ]:
learn.model

In [ ]:
learn.load_encoder('b2_enc')
learn.freeze()

In [ ]:
learn.lr_find()
learn.recorder.plot()

Here we follow the same training procedure as before. After 3 epochs on the training data, this model is performing better than the previous. This is entirely due to starting with a better language model.

In [ ]:
learn.fit_one_cycle(3, 2e-2, moms=(0.8,0.7))

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(3, slice(1e-2/(2.6**4),1e-2), moms=(0.8,0.7))

In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(3, slice(5e-3/(2.6**4),5e-3), moms=(0.8,0.7))

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(5, slice(1e-3/(2.6**4),1e-3), moms=(0.8,0.7))

In [ ]:
learn.fit_one_cycle(5, slice(1e-4/(2.6**4),1e-4), moms=(0.8,0.7))

In [ ]:
learn.save('coli_bact_pretrain')

# Test

In [ ]:
data_clas = GenomicTextClasDataBunch.from_df(path, train_df, test_df, tokenizer=tok, vocab=model_vocab,
                                            text_cols='Sequence', label_cols='Promoter', bs=300)
learn.data = data_clas

In [ ]:
get_scores(learn)

The bacterial ensemble pretrained model easily beats the E. coli genome pretrained model. The E. coli pretrained model trained to a validation loss of 0.24, while the ensemble pretrained model trained to a validation loss of 

Comparing this model to the E. coli pretrained model, accuracy increased from 91.9% to 97.3%, precision increased from 0.94 to 0.98, recall increased from 0.89 to 0.96, and MCC increased from 0.839 to 0.947.

The models were trained on the classification dataset for the same number of epochs. The increased performance shows the impact of a well trained language model on this technique. In addition to improved performance, the ensemble pretrained model overfit less and required less regularization (lower dropout).